In [1]:
%pip install transformer_lens
%pip install sae_lens
%pip install sae_vis
%pip install safetensors
%pip install circuitsvis


[notice] A new release of pip is available: 23.3.1 -> 24.0
[notice] To update, run: python3.11 -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.

[notice] A new release of pip is available: 23.3.1 -> 24.0
[notice] To update, run: python3.11 -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.

[notice] A new release of pip is available: 23.3.1 -> 24.0
[notice] To update, run: python3.11 -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.

[notice] A new release of pip is available: 23.3.1 -> 24.0
[notice] To update, run: python3.11 -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.

[notice] A new release of pip is available: 23.3.1 -> 24.0
[notice] To update, run: python3.11 -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [2]:
from transformer_lens import utils, HookedTransformer
model = HookedTransformer.from_pretrained(
    "gpt2-small",
    center_unembed=True,
    center_writing_weights=True,
    fold_ln=True,
    refactor_factored_attn_matrices=True,
)

/usr/local/lib/python3.11/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


Loaded pretrained model gpt2-small into HookedTransformer


In [3]:
import torch as t
from torch import Tensor
from typing import List, Union, Optional
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import numpy as np
import re
from typing import Dict
import pandas as pd
from jaxtyping import Float
import einops


def to_numpy(tensor):
    """
    Helper function to convert a tensor to a numpy array. Also works on lists, tuples, and numpy arrays.
    """
    if isinstance(tensor, np.ndarray):
        return tensor
    elif isinstance(tensor, (list, tuple)):
        array = np.array(tensor)
        return array
    elif isinstance(tensor, (t.Tensor, t.nn.parameter.Parameter)):
        return tensor.detach().cpu().numpy()
    elif isinstance(tensor, (int, float, bool, str)):
        return np.array(tensor)
    else:
        raise ValueError(f"Input to to_numpy has invalid type: {type(tensor)}")


# GENERIC PLOTTING FUNCTIONS

update_layout_set = {"xaxis_range", "yaxis_range", "hovermode", "xaxis_title", "yaxis_title", "colorbar", "colorscale", "coloraxis", "title_x", "bargap", "bargroupgap", "xaxis_tickformat", "yaxis_tickformat", "title_y", "legend_title_text", "xaxis_showgrid", "xaxis_gridwidth", "xaxis_gridcolor", "yaxis_showgrid", "yaxis_gridwidth", "yaxis_gridcolor", "showlegend", "xaxis_tickmode", "yaxis_tickmode", "margin", "xaxis_visible", "yaxis_visible", "bargap", "bargroupgap", "coloraxis_showscale", "xaxis_tickangle", "yaxis_scaleanchor", "xaxis_tickfont", "yaxis_tickfont"}

update_traces_set = {"textposition"}

def imshow(tensor: t.Tensor, renderer=None, **kwargs):
    kwargs_post = {k: v for k, v in kwargs.items() if k in update_layout_set}
    kwargs_pre = {k: v for k, v in kwargs.items() if k not in update_layout_set}
    if ("size" in kwargs_pre) or ("shape" in kwargs_pre):
        size = kwargs_pre.pop("size", None) or kwargs_pre.pop("shape", None)
        kwargs_pre["height"], kwargs_pre["width"] = size
    facet_labels = kwargs_pre.pop("facet_labels", None)
    border = kwargs_pre.pop("border", False)
    return_fig = kwargs_pre.pop("return_fig", False)
    text = kwargs_pre.pop("text", None)
    xaxis_tickangle = kwargs_post.pop("xaxis_tickangle", None)
    # xaxis_tickfont = kwargs_post.pop("xaxis_tickangle", None)
    static = kwargs_pre.pop("static", False)
    if "color_continuous_scale" not in kwargs_pre:
        kwargs_pre["color_continuous_scale"] = "RdBu"
    if "color_continuous_midpoint" not in kwargs_pre:
        kwargs_pre["color_continuous_midpoint"] = 0.0
    if "margin" in kwargs_post and isinstance(kwargs_post["margin"], int):
        kwargs_post["margin"] = dict.fromkeys(list("tblr"), kwargs_post["margin"])
    fig = px.imshow(to_numpy(tensor), **kwargs_pre).update_layout(**kwargs_post)
    if facet_labels:
        # Weird thing where facet col wrap means labels are in wrong order
        if "facet_col_wrap" in kwargs_pre:
            facet_labels = reorder_list_in_plotly_way(facet_labels, kwargs_pre["facet_col_wrap"])
        for i, label in enumerate(facet_labels):
            fig.layout.annotations[i]['text'] = label
    if border:
        fig.update_xaxes(showline=True, linewidth=1, linecolor='black', mirror=True)
        fig.update_yaxes(showline=True, linewidth=1, linecolor='black', mirror=True)
    if text:
        if tensor.ndim == 2:
            # if 2D, then we assume text is a list of lists of strings
            assert isinstance(text[0], list)
            assert isinstance(text[0][0], str)
            text = [text]
        else:
            # if 3D, then text is either repeated for each facet, or different
            assert isinstance(text[0], list)
            if isinstance(text[0][0], str):
                text = [text for _ in range(len(fig.data))]
        for i, _text in enumerate(text):
            fig.data[i].update(
                text=_text,
                texttemplate="%{text}",
                textfont={"size": 12}
            )
    # Very hacky way of fixing the fact that updating layout with xaxis_* only applies to first facet by default
    if xaxis_tickangle is not None:
        n_facets = 1 if tensor.ndim == 2 else tensor.shape[0]
        for i in range(1, 1+n_facets):
            xaxis_name = "xaxis" if i == 1 else f"xaxis{i}"
            fig.layout[xaxis_name]["tickangle"] = xaxis_tickangle
    return fig if return_fig else fig.show(renderer=renderer, config={"staticPlot": static})


def reorder_list_in_plotly_way(L: list, col_wrap: int):
    '''
    Helper function, because Plotly orders figures in an annoying way when there's column wrap.
    '''
    L_new = []
    while len(L) > 0:
        L_new.extend(L[-col_wrap:])
        L = L[:-col_wrap]
    return L_new


def line(y: Union[t.Tensor, List[t.Tensor]], renderer=None, **kwargs):
    '''
    Edit to this helper function, allowing it to take args in update_layout (e.g. yaxis_range).
    '''
    kwargs_post = {k: v for k, v in kwargs.items() if k in update_layout_set}
    kwargs_pre = {k: v for k, v in kwargs.items() if k not in update_layout_set}
    if ("size" in kwargs_pre) or ("shape" in kwargs_pre):
        size = kwargs_pre.pop("size", None) or kwargs_pre.pop("shape", None)
        kwargs_pre["height"], kwargs_pre["width"] = size
    return_fig = kwargs_pre.pop("return_fig", False)
    if "margin" in kwargs_post and isinstance(kwargs_post["margin"], int):
        kwargs_post["margin"] = dict.fromkeys(list("tblr"), kwargs_post["margin"])
    if "xaxis_tickvals" in kwargs_pre:
        tickvals = kwargs_pre.pop("xaxis_tickvals")
        kwargs_post["xaxis"] = dict(
            tickmode = "array",
            tickvals = kwargs_pre.get("x", np.arange(len(tickvals))),
            ticktext = tickvals
        )
    if "hovermode" not in kwargs_post:
        kwargs_post["hovermode"] = "x unified"
    if "use_secondary_yaxis" in kwargs_pre and kwargs_pre["use_secondary_yaxis"]:
        del kwargs_pre["use_secondary_yaxis"]
        if "labels" in kwargs_pre:
            labels: dict = kwargs_pre.pop("labels")
            kwargs_post["yaxis_title_text"] = labels.get("y1", None)
            kwargs_post["yaxis2_title_text"] = labels.get("y2", None)
            kwargs_post["xaxis_title_text"] = labels.get("x", None)
        for k in ["title", "template", "width", "height"]:
            if k in kwargs_pre:
                kwargs_post[k] = kwargs_pre.pop(k)
        fig = make_subplots(specs=[[{"secondary_y": True}]]).update_layout(**kwargs_post)
        y0 = to_numpy(y[0])
        y1 = to_numpy(y[1])
        x0, x1 = kwargs_pre.pop("x", [np.arange(len(y0)), np.arange(len(y1))])
        name0, name1 = kwargs_pre.pop("names", ["yaxis1", "yaxis2"])
        fig.add_trace(go.Scatter(y=y0, x=x0, name=name0), secondary_y=False)
        fig.add_trace(go.Scatter(y=y1, x=x1, name=name1), secondary_y=True)
    else:
        y = list(map(to_numpy, y)) if isinstance(y, list) and not (isinstance(y[0], int) or isinstance(y[0], float)) else to_numpy(y)
        names = kwargs_pre.pop("names", None)
        fig = px.line(y=y, **kwargs_pre).update_layout(**kwargs_post)
        if names is not None:
            fig.for_each_trace(lambda trace: trace.update(name=names.pop(0)))
    return fig if return_fig else fig.show(renderer=renderer)


def scatter(x, y, renderer=None, **kwargs):
    x = to_numpy(x)
    y = to_numpy(y)
    add_line = None
    if "add_line" in kwargs:
        add_line = kwargs.pop("add_line")
    kwargs_post = {k: v for k, v in kwargs.items() if k in update_layout_set}
    kwargs_traces = {k: v for k, v in kwargs.items() if k in update_traces_set}
    kwargs_pre = {k: v for k, v in kwargs.items() if k not in (update_layout_set | update_traces_set)}
    if ("size" in kwargs_pre) or ("shape" in kwargs_pre):
        size = kwargs_pre.pop("size", None) or kwargs_pre.pop("shape", None)
        kwargs_pre["height"], kwargs_pre["width"] = size
    return_fig = kwargs_pre.pop("return_fig", False)
    facet_labels = kwargs_pre.pop("facet_labels", None)
    if "margin" in kwargs_post and isinstance(kwargs_post["margin"], int):
        kwargs_post["margin"] = dict.fromkeys(list("tblr"), kwargs_post["margin"])
    fig = px.scatter(y=y, x=x, **kwargs_pre).update_layout(**kwargs_post)
    if add_line is not None:
        xrange = fig.layout.xaxis.range or [x.min(), x.max()]
        yrange = fig.layout.yaxis.range or [y.min(), y.max()]
        add_line = add_line.replace(" ", "")
        if add_line in ["x=y", "y=x"]:
            fig.add_trace(go.Scatter(mode='lines', x=xrange, y=xrange, showlegend=False))
        elif re.match("(x|y)=", add_line):
            try: c = float(add_line.split("=")[1])
            except: raise ValueError(f"Unrecognized add_line: {add_line}. Please use either 'x=y' or 'x=c' or 'y=c' for some float c.")
            x, y = ([c, c], yrange) if add_line[0] == "x" else (xrange, [c, c])
            fig.add_trace(go.Scatter(mode='lines', x=x, y=y, showlegend=False))
        else:
            raise ValueError(f"Unrecognized add_line: {add_line}. Please use either 'x=y' or 'x=c' or 'y=c' for some float c.")
    if facet_labels:
        for i, label in enumerate(facet_labels):
            fig.layout.annotations[i]['text'] = label
    fig.update_traces(**kwargs_traces)
    return fig if return_fig else fig.show(renderer=renderer)


def bar(tensor, renderer=None, **kwargs):
    '''
    '''
    if isinstance(tensor, list):
        if isinstance(tensor[0], t.Tensor): arr = [to_numpy(tn) for tn in tensor]
        elif isinstance(tensor[0], list): arr = [np.array(tn) for tn in tensor]
        else: arr = np.array(tensor)
    else:
        arr = to_numpy(tensor)
    kwargs_post = {k: v for k, v in kwargs.items() if k in update_layout_set}
    kwargs_pre = {k: v for k, v in kwargs.items() if k not in update_layout_set}
    return_fig = kwargs_pre.pop("return_fig", False)
    names = kwargs_pre.pop("names", None)
    if "hovermode" not in kwargs_post:
        kwargs_post["hovermode"] = "x unified"
    if "margin" in kwargs_post and isinstance(kwargs_post["margin"], int):
        kwargs_post["margin"] = dict.fromkeys(list("tblr"), kwargs_post["margin"])
    fig = px.bar(y=arr, **kwargs_pre).update_layout(**kwargs_post)
    if names is not None:
        for i in range(len(fig.data)):
            fig.data[i]["name"] = names[i // 2 if "marginal" in kwargs_pre else i]
    return fig if return_fig else fig.show(renderer=renderer)


def hist(tensor, renderer=None, **kwargs):
    kwargs_post = {k: v for k, v in kwargs.items() if k in update_layout_set}
    kwargs_pre = {k: v for k, v in kwargs.items() if k not in update_layout_set}
    draw = kwargs_pre.pop("draw", True)
    static = kwargs_pre.pop("static", False)
    return_fig = kwargs_pre.pop("return_fig", False)
    if isinstance(tensor, list):
        if isinstance(tensor[0], t.Tensor): arr = [to_numpy(tn) for tn in tensor]
        elif isinstance(tensor[0], list): arr = [np.array(tn) for tn in tensor]
        else: arr = np.array(tensor)
    else:
        arr = to_numpy(tensor)
    if "modebar_add" not in kwargs_post:
        kwargs_post["modebar_add"] = ['drawline', 'drawopenpath', 'drawclosedpath', 'drawcircle', 'drawrect', 'eraseshape']
    add_mean_line = kwargs_pre.pop("add_mean_line", False)
    names = kwargs_pre.pop("names", None)
    if "barmode" not in kwargs_post:
        kwargs_post["barmode"] = "overlay"
    if "bargap" not in kwargs_post:
        kwargs_post["bargap"] = 0.0
    if "margin" in kwargs_post and isinstance(kwargs_post["margin"], int):
        kwargs_post["margin"] = dict.fromkeys(list("tblr"), kwargs_post["margin"])
    if "hovermode" not in kwargs_post:
        kwargs_post["hovermode"] = "x unified"
    if "autosize" not in kwargs_post:
        kwargs_post["autosize"] = False

    # If `arr` has a list of arrays, then just doing px.histogram doesn't work annoyingly enough
    # This is janky, even for my functions!
    if isinstance(arr, list) and isinstance(arr[0], np.ndarray):
        assert "marginal" not in kwargs_pre, "Can't use `marginal` with a list of arrays"
        for thing_to_move_from_pre_to_post in ["title", "template", "height", "width", "labels"]:
            if thing_to_move_from_pre_to_post in kwargs_pre:
                kwargs_post[thing_to_move_from_pre_to_post] = kwargs_pre.pop(thing_to_move_from_pre_to_post)
        if "labels" in kwargs_post:
            kwargs_post["xaxis_title_text"] = kwargs_post["labels"].get("x", "")
            kwargs_post["yaxis_title_text"] = kwargs_post["labels"].get("y", "")
            del kwargs_post["labels"]
        fig = go.Figure(layout=go.Layout(**kwargs_post))
        if "nbins" in kwargs_pre:
            kwargs_pre["nbinsx"] = int(kwargs_pre.pop("nbins"))
        for x in arr:
            fig.add_trace(go.Histogram(x=x, name=names.pop(0) if names is not None else None, **kwargs_pre))
    else:
        fig = px.histogram(x=arr, **kwargs_pre).update_layout(**kwargs_post)
        if names is not None:
            for i in range(len(fig.data)):
                fig.data[i]["name"] = names[i // 2 if "marginal" in kwargs_pre else i]

    if add_mean_line:
        if arr.ndim == 1:
            fig.add_vline(x=arr.mean(), line_width=3, line_dash="dash", line_color="black", annotation_text=f"Mean = {arr.mean():.3f}", annotation_position="top")
        elif arr.ndim == 2:
            for i in range(arr.shape[0]):
                fig.add_vline(x=arr[i].mean(), line_width=3, line_dash="dash", line_color="black", annotation_text=f"Mean = {arr.mean():.3f}", annotation_position="top")
    return fig if return_fig else fig.show(renderer=renderer)


In [4]:
_, cache = model.run_with_cache("a short story")
tokens = model.to_str_tokens("a short story")
print(tokens)

['<|endoftext|>', 'a', ' short', ' story']


In [5]:
import circuitsvis as cv

print(type(cache))
attention_pattern = cache["pattern", 0][0]
print(attention_pattern.shape)
str_tokens = model.to_str_tokens("a short story")

print("Layer 0 Head Attention Patterns:")
display(cv.attention.attention_patterns(
    tokens=str_tokens,
    attention=attention_pattern,
    # attention_head_names=[f"L0H{i}" for i in range(12)],
))

<class 'transformer_lens.ActivationCache.ActivationCache'>
torch.Size([12, 4, 4])
Layer 0 Head Attention Patterns:


We will use the function cv.attention.attention_patterns, which takes two important arguments:

attention: Attention head activations.
This should be a tensor of shape [nhead, seq_dest, seq_src], i.e. the [i, :, :]th element is the grid of attention patterns (probabilities) for the ith attention head.
We get this by indexing our gpt2_cache object.

In [22]:
_, cache = model.run_with_cache("Mary and John went to the store. John gave the cart to")
tokens = model.to_str_tokens("Mary and John went to the store. John gave the cart to")
print(tokens)

batch = 0
print(cache["pattern", 0].shape)

activations = []

## Get all the activations for every head in every layer
for layer in range(model.cfg.n_layers):
  # print(f"layer {layer}")
  layer_activations = []
  for head in range(model.cfg.n_heads):
    # print(f"head {head}")
    head_acts = cache["pattern", layer][batch][head][1:, 1:] # getting the attention patterns for everything but the first <|endoftext|> token

    layer_activations.append(head_acts)
  layer_activations = t.stack(layer_activations)
  activations.append(layer_activations)

activations = t.stack(activations)

print(activations.shape)

['<|endoftext|>', 'Mary', ' and', ' John', ' went', ' to', ' the', ' store', '.', ' John', ' gave', ' the', ' cart', ' to']
torch.Size([1, 12, 14, 14])
torch.Size([12, 12, 13, 13])


In [31]:
print(tokens)
print(len(tokens))

def get_attention_layer_activations(position=-1):
  position_contributions =  t.zeros(len(tokens[1:]), device=model.cfg.device)

  for layer in range(model.cfg.n_layers):
    layer_activations = activations[layer]
    # display(cv.attention.attention_patterns(
    #   tokens=tokens[1:],
    #   attention=layer_activations,
    #   # attention_head_names=[f"L0H{i}" for i in range(12)],
    # ))

    # print(layer_activations.shape)
    token_activations = layer_activations[:,position,:]
    # print(token_activations.shape)
    summed_values = einops.reduce(token_activations, 'tensors num -> num', 'sum')
    # print(f"summed values: {len(summed_values)}")
    normalized_tensor = summed_values / t.sum(summed_values)
    # print(normalized_tensor)

    position_contributions += normalized_tensor

  position_contributions = position_contributions/model.cfg.n_layers
  return position_contributions

total_position_contributions =  t.zeros((len(tokens) - 1, len(tokens[1:])), device=model.cfg.device)

for i in range(len(tokens) - 1):
  position_contributions = get_attention_layer_activations(i)
  total_position_contributions[i] = t.tensor(position_contributions, device=model.cfg.device)

total_position_contributions.cpu().numpy()

print(total_position_contributions.shape)
print(total_position_contributions)

for i, token in enumerate(tokens[1:]):
  print(f"Token {i}: {token}")

imshow(
    total_position_contributions,
    labels={"x": "Sequence position", "y": "Token position"},
    title="Token contributions",
    text_auto=".2f",
    width=900, height=600
)

['<|endoftext|>', 'Mary', ' and', ' John', ' went', ' to', ' the', ' store', '.', ' John', ' gave', ' the', ' cart', ' to']
14


/var/folders/ny/hc9c812905l118ss_x5j9x380000gn/T/ipykernel_25228/3840749762.py:32: UserWarning:

To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).



torch.Size([13, 13])
tensor([[1.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
         0.0000, 0.0000, 0.0000, 0.0000],
        [0.6591, 0.3409, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
         0.0000, 0.0000, 0.0000, 0.0000],
        [0.2722, 0.4561, 0.2717, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
         0.0000, 0.0000, 0.0000, 0.0000],
        [0.1685, 0.2721, 0.2381, 0.3213, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
         0.0000, 0.0000, 0.0000, 0.0000],
        [0.1234, 0.1405, 0.1601, 0.3444, 0.2315, 0.0000, 0.0000, 0.0000, 0.0000,
         0.0000, 0.0000, 0.0000, 0.0000],
        [0.0894, 0.0987, 0.1007, 0.1867, 0.3586, 0.1659, 0.0000, 0.0000, 0.0000,
         0.0000, 0.0000, 0.0000, 0.0000],
        [0.0607, 0.0621, 0.0586, 0.1601, 0.2284, 0.2136, 0.2164, 0.0000, 0.0000,
         0.0000, 0.0000, 0.0000, 0.0000],
        [0.1507, 0.0831, 0.1294, 0.1061, 0.1053, 0.0793, 0.1472, 0.1988, 0.0000,
         0.0000, 0.0000, 0.0000, 0.00

In [46]:
print(total_position_contributions.shape)

# Convert the tensor to a numpy array

print(len(tokens[1:]))
labels = tokens[1:]
print(tokens[1:])

# # Create a list of labels for the x and y axes
# num_tokens = total_position_contributions.shape[0]
# labels = [f"Token {i+1}" for i in range(num_tokens)]
labels = [re.sub(r'[^\w]', '_', token) for token in labels]
tokens_list = ['Mary', ' and', ' John', ' went', ' to', ' the', ' store', '.', ' John', ' gave', ' the', ' cart', ' to']
num_tokens = len(tokens_list)

# Replace spaces and special characters with underscores
labels = [re.sub(r'[^\w]', '_', token) for token in tokens_list]
print(labels)
print(len(labels))

# Create a figure with Plotly Express
fig = px.imshow(total_position_contributions.cpu().numpy(), x=labels, y=labels, color_continuous_scale='Viridis')

# Update the layout
fig.update_layout(
    title='Information flow',
    xaxis_title='Source Tokens',
    yaxis_title='Destination Tokens',
    font=dict(size=12)
)

# Show the figure
fig.show()

torch.Size([13, 13])
13
['Mary', ' and', ' John', ' went', ' to', ' the', ' store', '.', ' John', ' gave', ' the', ' cart', ' to']
['Mary', '_and', '_John', '_went', '_to', '_the', '_store', '_', '_John', '_gave', '_the', '_cart', '_to']
13


In [53]:
import plotly.graph_objects as go

def heatmap(matrix, x_labels, y_labels, title):
    attention_matrix = total_position_contributions.cpu().numpy()
    # tokens_list = ['Mary', ' and', ' John', ' went', ' to', ' the', ' store', '.', ' John', ' gave', ' the', ' cart', ' to']
    # num_tokens = len(tokens_list)

    # Create the heatmap trace
    trace = go.Heatmap(
        z=matrix,
        x0=0, dx=1,
        y0=0, dy=1,
        xgap=1, ygap=1,
        xaxis='x',
        yaxis='y'
    )

    # Create the figure and set the x and y labels
    fig = go.Figure(data=trace)
    fig.update_layout(
        title=title,
        xaxis=dict(
            tickmode='array',
            tickvals=list(range(len(x_labels))),

            ticktext=x_labels,
            tickfont=dict(size=12)
        ),
        yaxis=dict(
            tickmode='array',
            tickvals=list(range(len(y_labels))),
            ticktext=y_labels,
            tickfont=dict(size=12)
        )
    )
    # Show the figure
    fig.show()

In [54]:
# Aggregate all attention outputs 
def get_activations(prompt):
    _, cache = model.run_with_cache(prompt)
    batch = 0

    activations = []

    ## Get all the activations for every head in every layer
    for layer in range(model.cfg.n_layers):
        layer_activations = []
        for head in range(model.cfg.n_heads):
            head_acts = cache["pattern", layer][batch][head][1:, 1:] # getting the attention patterns for eveall tokens except the first <|endoftext|> token

            layer_activations.append(head_acts)
        layer_activations = t.stack(layer_activations)
        activations.append(layer_activations)

    activations = t.stack(activations)

    print(activations.shape)
    return activations

def get_attention_layer_activations(position=-1):
  position_contributions =  t.zeros(len(tokens[1:]), device=model.cfg.device)

  for layer in range(model.cfg.n_layers):
    layer_activations = activations[layer]
    
    token_activations = layer_activations[:,position,:]
    summed_values = einops.reduce(token_activations, 'tensors num -> num', 'sum')
    normalized_tensor = summed_values / t.sum(summed_values)

    position_contributions += normalized_tensor

  position_contributions = position_contributions/model.cfg.n_layers
  return position_contributions

def plot_info_flow(prompt):
  tokens = model.to_str_tokens(prompt)
  activations = get_activations(prompt)

  total_position_contributions =  t.zeros((len(tokens) - 1, len(tokens[1:])), device=model.cfg.device)

  for i in range(len(tokens) - 1):
    position_contributions = get_attention_layer_activations(i)
    total_position_contributions[i] = t.tensor(position_contributions, device=model.cfg.device)

  heatmap(total_position_contributions.cpu().numpy(), tokens[1:], tokens[1:], "Information flow")

plot_info_flow("Mary and John went to the store. John gave the cart to")


torch.Size([12, 12, 13, 13])


/var/folders/ny/hc9c812905l118ss_x5j9x380000gn/T/ipykernel_25228/4050287523.py:56: UserWarning:

To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).

